# 🧠 LoRA Inference - Stable Diffusion

This notebook demonstrates how to load a trained LoRA adapter into the Stable Diffusion v1.4 pipeline and generate stylized images from prompts.

## Step 1: Install & Import Dependencies

In [ ]:
# %pip install torch==2.1.2+cu118 --index-url https://download.pytorch.org/whl/cu118
# %pip install transformers==4.36.2
# %pip install diffusers==0.25.0
# %pip install peft==0.7.1
# %pip install huggingface_hub==0.20.3


In [ ]:

import torch
from diffusers import StableDiffusionPipeline
from peft import PeftModel
from PIL import Image
import os

## Step 2: Configuration

In [ ]:
base_model_id = "CompVis/stable-diffusion-v1-4"
lora_weights_path = "./lora_output"
output_dir = "./outputs"
os.makedirs(output_dir, exist_ok=True)

prompts = [
    "AI robot teaching students in a village school",
    "A traditional Gurukul reimagined with VR and AR",
    "Students learning under a hologram tree",
    "A child meditating in a virtual garden",
    "Knowledge as light in a dark classroom"
]

## Step 3: Load Base Pipeline

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_id,
    torch_dtype=torch.float32
).to("cuda")

## Step 4: Inject LoRA Adapter

In [ ]:
pipe.unet = PeftModel.from_pretrained(pipe.unet, lora_weights_path)
pipe.unet = pipe.unet.to(torch.float32)

## Step 5: Run Inference and Save Images

In [ ]:
for i, prompt in enumerate(prompts):
    print(f"🔹 Generating image for: {prompt}")
    image = pipe(prompt).images[0]
    image.save(os.path.join(output_dir, f"image_{i+1}.png"))

print(f"\n✅ All images saved to: {output_dir}")

## Step 6: Display Images Inline

In [ ]:
from IPython.display import display

for i in range(1, len(prompts)+1):
    img_path = os.path.join(output_dir, f"image_{i}.png")
    if os.path.exists(img_path):
        display(Image.open(img_path))